In [88]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
import os

In [89]:
loader = DirectoryLoader("/Users/hritik/Desktop/Grading Doc RAG/Document", glob="**/*.html",loader_cls=UnstructuredHTMLLoader)
docs = loader.load()
len(docs)

166

In [90]:
dotenv_path = '../.env'
load_dotenv(dotenv_path)
HUGGINGFACEHUB_API_TOKEN = os.getenv("HF_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGINGFACEHUB_API_TOKEN, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

In [91]:
db = Chroma.from_documents(docs, embeddings, persist_directory='vectordb/chroma/')

OperationalError: attempt to write a readonly database

In [92]:
print(db._collection.count())

203


In [93]:
ques = "paradox"

In [94]:
docs = db.similarity_search(ques,k=3)

In [95]:
docs

[Document(page_content='Paradox\'22\n\n\n\nPrevious\n\nNext\n\nSports\n\nInteractive Sessions\n\nCultural Events\n\nWorkshops\n\nCompany Discussions\n\nSponsors\n\nPARADOX\'22\n\nStudent Festival\n\nOrganised by the students of IITM BS Degree Program at the IIT Madras Campus\n\n20th, 21st & 22nd May 2022\n\n1500+ Programming & Data Science students of IIT Madras attended the first on-campus three day\n                festival, which was fully curated and organised by the students. This was a huge platform for\n                interaction between fellow students, faculty, industry experts, and companies.\n\nParadox \'22 also marked the formal distribution of provisional diploma certificates to 101 students,\n                including 17 students from families of income less than ₹1 lakh per annum.\n\nSports\n\nPrevious\n\nNext\n\nAs an icebreaker and warm-up to the three day festival, a whole series of games (chess, table\n        tennis, box cricket, football 5s, etc.) were organised w